In [1]:
import folium
import numpy as np
from folium import plugins

In [2]:
from rlway.pyosrd import OSRD

In [3]:
france = OSRD('../data', infra_json='france.id15.railjson.json')

In [4]:
line_names = [
    (t['extensions']['sncf']['line_name']) 
    for t in france.infra['track_sections']
]
line_codes = [
    (t['extensions']['sncf']['line_code']) 
    for t in france.infra['track_sections']
]

In [5]:
import pandas as pd

df = pd.DataFrame({
    'name': line_names,
    'code': line_codes
}).drop_duplicates().reset_index(drop=True)

In [14]:
df[df['name'].str.contains('Bordeaux')]

,name,code
9,Ligne de Paris-Austerlitz à Bordeaux-St-Jean,570000
10,Ligne de Chartres à Bordeaux-St-Jean,500000
44,Ligne de Bordeaux-St-Jean à Sète-Ville,640000
58,Ligne de Bordeaux-St-Jean à Irun,655000
428,Raccordement de Tours à Monts (bifurcation de ...,564300
441,Ceinture de Bordeaux,586000
500,Ligne de Lormont à Bordeaux-Bastide,568000
566,Raccordement circulaire de Bordeaux-St-Jean,640306
763,Ligne de Bordeaux-Bénauge à La Sauvetat-du-Dropt,637000


In [7]:
station_names = {
    station['id']: station['extensions']['identifier']['name']
    for station in france.infra['operational_points']
}

In [8]:
from typing import Dict, Any, Union, List

def filter_by_line_codes(infra: Dict[str, Any], codes: Union[int, List[int]]) -> Dict[str, Any]:

    if not isinstance(codes, list):
        codes = [codes]

    subinfra = infra.copy()

    # 'version'
    subinfra["version"] = infra['version']
    
    # 'track_sections'
    subinfra['track_sections'] = [
        track for track in infra['track_sections'] if track['extensions']['sncf']['line_code'] in codes
    ]

    track_section_ids = [track['id'] for track in subinfra['track_sections']]

    # 'signals'
    subinfra['signals'] = [
        signal for signal in infra['signals'] if signal['track'] in track_section_ids
    ]

    # 'speed_sections'


    # 'detectors'
    subinfra['detectors'] = [
        detector for detector in infra['detectors'] if detector['track'] in track_section_ids
    ]
    detector_ids = [detector['id'] for detector in subinfra['detectors']]

    # 'track_section_links'
    subinfra['track_section_links'] = [
        link for link in infra['track_section_links']
        if link['src']['track'] in track_section_ids or link['src']['track'] in track_section_ids
    ]

    # 'switches'
    # TODO: filter switches
    subinfra['switches'] = infra['switches']

    # 'switch_types'
    subinfra['switch_types'] = infra['switch_types']

    # 'buffer_stops'
    subinfra['buffer_stops'] = [
        buffer_stop
        for buffer_stop in infra['buffer_stops']
        if buffer_stop['track'] in track_section_ids
    ]

    # 'routes'


    # 'operational_points'
    op_ids = set()
    for op in infra['operational_points']:
        for track in [part['track'] for part in op['parts']]:
            if track in track_section_ids:
                op_ids.add(op['id'])

    subinfra['operational_points'] = [
        op
        for op in infra['operational_points']
        if op['id'] in op_ids
    ]

    # 'catenaries'


    # 'dead_sections'

    return subinfra

In [15]:
infra = filter_by_line_codes(france.infra, 570000)


In [16]:
{
    station['id']: station['extensions']['identifier']['name']
    for station in infra['operational_points']
}

{'d9c64ea2-6667-11e3-89ff-01f464e0362d': 'St-Pierre-des-Corps',
 'fd21a670-2fae-11e5-a0ff-017864e0362d': 'Monts-Sud',
 'd9a66ab0-6667-11e3-89ff-01f464e0362d': 'Grand-Pont-Preuilly',
 'd9d63344-6667-11e3-89ff-01f464e0362d': 'Vitry-sur-Seine',
 'd986929e-6667-11e3-89ff-01f464e0362d': 'Ingrandes-sur-Vienne',
 'd98634d8-6667-11e3-89ff-01f464e0362d': 'Beaugency',
 'd9c9441c-6667-11e3-89ff-01f464e0362d': 'Choisy-le-Roi',
 'd97041ec-6667-11e3-89ff-01f464e0362d': 'Châtellerault',
 'd9894eea-6667-11e3-89ff-01f464e0362d': 'Paris-Austerlitz',
 'd9a67140-6667-11e3-89ff-01f464e0362d': 'Montmoreau',
 'd99c73d2-6667-11e3-89ff-01f464e0362d': 'Bibliothèque-François-Mitterrand',
 'd9c0ab3e-6667-11e3-89ff-01f464e0362d': 'Angerville',
 'd98917ae-6667-11e3-89ff-01f464e0362d': 'Maillé',
 'd94c499c-6667-11e3-89ff-01f464e0362d': 'Bibliothèque-François-Mitterrand',
 'd9467cc8-6667-11e3-89ff-01f464e0362d': 'Dissay (Vienne)',
 'd9e0d910-6667-11e3-89ff-01f464e0362d': 'Juvisy',
 'd9d857b6-6667-11e3-89ff-01f464e036

In [17]:
# track_section_coordinates = {
#     ts['id']: [(point[1], point[0]) for point in ts['geo']['coordinates']]
#     for ts in infra['track_sections']
# }

# m = folium.Map(width='50%', height='50%')

# tracks = folium.FeatureGroup(name='small_infra')
# for id, line in track_section_coordinates.items():
#     folium.PolyLine(line, tooltip=id, color='black').add_to(tracks)
# tracks.add_to(m)

# m.fit_bounds(tracks.get_bounds())

# folium.LayerControl().add_to(m)

# m.add_child(plugins.Fullscreen())

# # m

In [18]:
from rlway.pyosrd.viz.map import folium_map

line = OSRD('./line')
line.infra = infra

In [19]:
folium_map(line)

KeyError: '3711a556-19be-11e5-a6ff-01fc64e0362d'